1. let's import all of the packages we'll need!
- I make usage of the package "neurokin" a lot. To install it, check out the documentation website : https://neurokin.readthedocs.io/en/latest/index.html
- under "neurokin package" you can also see more about a lot of the functions im using here and you can check out the neurokin GitHub repository to see more documentation etc.!!

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import sys
from Demos.BackupSeek_streamheaders import stream_name
sys.path.append("../neurokin/")
from neurokin.utils.neural import processing, importing, neural_plot
from neurokin.utils.experiments import neural_states_helper, neural_correlates_plot, spider_factory
from neurokin.locomotion_states import NeuralCorrelatesStates
import json
from scipy.stats import mannwhitneyu, normaltest, ttest_ind, levene, ttest_rel, shapiro
from scipy import signal
from matplotlib.ticker import FormatStrFormatter
from matplotlib.lines import Line2D
from neurokin.utils.experiments import neural_states_helper, neural_correlates_plot, spider_factory
from neurokin.locomotion_states import NeuralCorrelatesStates
from neurokin.neural_data import NeuralData

Filename: C:\Users\User\AppData\Local\Temp\bkrFB18.tmp

Type: 3 Security descriptor data Attributes: 2 Size: 164 Name len: 0
Name:Unnamed
Moved:  164

Type: 1 Standard data Attributes: 0 Size: 116 Name len: 0
Name:Unnamed
Moved:  116

Type: 4 Alternative data streams Attributes: 0 Size: 168 Name len: 52
Name::SummaryInformation:$DATA
Moved:  168

Type: 4 Alternative data streams Attributes: 0 Size: 200 Name len: 40
Name::anotherstream:$DATA
Moved:  200

Type: 4 Alternative data streams Attributes: 0 Size: 132 Name len: 34
Name::streamdata:$DATA
Moved:  132

Type: 4 Alternative data streams Attributes: 0 Size: 0 Name len: 90
Name::{4c8cc155-6c1e-11d1-8e41-00c04fb9386d}:$DATA


2. Then, we get the data we want to analyze and further define some important parameters:
- we can define which animals are parkinsonian and which ones are healthy. We can also define if we want to skip some animals (for example after exclusion analysis)
- we also create a dictionary that tells what channel is the right one for each animal

In [2]:
experiment_path = "../../data_labrotation/dataset_EES_OL_fall"

In [3]:
NFFT = 2**12
NOV = int(NFFT/4)
TIME_CUTOFF = 1.5


pda = ["NWE00180", "NWE00186", "NWE00187", "NWE00191", "NWE00192", "NWE00193"] 
healthy = ["NWE00188", "NWE00189", "NWE00197"]
skip_animals = []

CHANNEL_DICT = {"NWE00180": 3,
                "NWE00186": 3,
                "NWE00187": 3,
                "NWE00188": 3,
                "NWE00189": 3,
                "NWE00191": 3,
                "NWE00192": 3,
                "NWE00193": 3,
                "NWE00197": 3,
                "NWE00198": 3}

3. Here, we create our experiment structure and define, what stimulation conditions we have:

In [4]:
experiment_structure_path = experiment_path + "/experiment_structure_fall.yaml"
conditions = ["baseline", "cervical40", "cervical125", "lumbar40", "lumbar125"]  

4. Here, firstly an instance of the class "NeuralCorrelatesStates" is created --> ncs = an object. This processes and organizes the neural data according to the parameters that are passed (e.g. timeslice etc.)
- With the next two steps, we create a dataframe containing data of each trial and timestamps of events (Akinesia, stationary movement, gait) and then we save it
- This dataframe contains information on the date, the condition etc.
- If we re-run the code, we can just call the priorly saved dataframe (the commented last line of the code)

In [ ]:
ncs = NeuralCorrelatesStates(timeslice=TIME_CUTOFF,                                                                          experiment_structure_filepath=experiment_structure_path,
                            skip_subjects=skip_animals, skip_conditions=[])

ncs.create_events_dataset(experiment_path, verbose=False)
ncs.save_dataset("events_dataset", "./event_dataset")
#ncs.events_dataset = pd.read_pickle("./event_dataset.pkl")

In [ ]:
ncs.events_dataset.head(10) # check out how the dataframe looks!

5. Additionally, we create pandas dataframe that contains neural data that corresponds to different locomotor events in the runs. 
(= what happened in the brain while the animal was in Akinesia? Or while in gait or stationary movement?) and save it. 
We also define the sampling frequency of the recording. Again, we can call the priorly saved dataframe when re-running the code.

In [ ]:
ncs.create_raw_neural_dataset(experiment_path, 
                                 stream_names=["LFP1", "NPr1", "EOG1"],
                                  ch_of_interest=CHANNEL_DICT)
ncs.save_dataset("raw_neural_correlates_dataset", "./raw_neural_correlates_dataset")
# ncs.raw_neural_correlates_dataset = pd.read_pickle("./raw_neural_correlates_dataset.pkl")
ncs.fs= 24414.1

In [ ]:
ncs.raw_neural_correlates_dataset.head(5) # check out how the dataframe looks!

6. Next, "create_psd_dataset" computes the Power Spectra Density of the neural correlates. "Plot_prep_psds_dataset" then creates a reduced dataframe (=df) of the Power Spectral Density ready for us to plot the PSD plot : 

In [ ]:
ncs.create_psd_dataset(NFFT, NOV, zscore=True)

In [ ]:
df = ncs.plot_prep_psds_dataset(test_sbj_list=pda, condense=True)

In [ ]:
df.head(5) # check out how the dataframe looks!

7. Then, we define the freqs, which is the Frequency array from the Fourier Transform : 

In [ ]:
ncs.freqs = np.fft.rfftfreq(NFFT, d=1 / ncs.fs)

8. If we want to first take a look at all the animals on their own, we plot all the animals in Baseline (e.g. to see if a PD animal didn't show a strong beta band)  :

In [ ]:
df_only_baseline = df[df["condition"] == "baseline"] # taking only the baseline condition!

Then lets plot them : 
- We loop through our axes (= three plots for the three states) and for each state we loop through the animals. If they are healthy, they are plotted in a gray-tone if not, they are visualized in a red-tone.

In [ ]:
max_freq = 100
min_freq = 5

idx_max = processing.find_closest_index(ncs.freqs, max_freq)  # storing index that represents closest frequency to max_frequency in data
idx_min = processing.find_closest_index(ncs.freqs, min_freq)

fig, axs = plt.subplots(1, 3, sharey=True, figsize=(10, 4))
fig.subplots_adjust(wspace=0)
sns.set_style("white")
states = ["event_fog", "event_nlm", "event_gait"]
titles = ["Akinesia", "Stationary \n movement", "Gait"]
animals = df_only_baseline["subject"].unique()
gray_palette = sns.color_palette("Greys", n_colors=len(healthy) + 1)
red_palette = sns.color_palette("Reds", n_colors=len(pda) + 1)

for i, ax in enumerate(axs):
    state = states[i]
    for j, animal in enumerate(animals):
        animal_data = df_only_baseline[(df_only_baseline["subject"] == animal)][state].values[0]
        f = ncs.freqs[idx_min:idx_max]
        animal_psd = animal_data[idx_min:idx_max]
        if animal in healthy:
            gray_index = healthy.index(animal)
            color_palette = gray_palette[gray_index + 1]
        else:
            red_index = pda.index(animal)
            color_palette = red_palette[red_index + 1]
        
        ax.plot(f, animal_psd, color=color_palette, linewidth=3, label = animal, alpha=0.8)
        
    ax.spines[['left']].set_visible(False)

ylim = 20

axs[0].set_ylabel("PSD [AU]", fontsize=15)
axs[0].set_ylim(0, ylim)
axs[1].set_xlabel("Freqs [Hz]", fontsize=15)
for a in axs:
    a.spines[['right', 'top']].set_visible(False)
plt.legend()   
plt.tight_layout()
#plt.savefig("Baseline_PSD_test.png")

9. To plot the beta band box plot, first we condense a priorly created dataframe (for each locomotor state we see the timestamps from when to when in a run the animal was in that state).
Then we also create an empty dataframe called "beta_df" where we already have the three states as columns : 

In [ ]:
df = ncs.psds_correlates_dataset.copy()
df = neural_states_helper.condense_distribution_event_types(df)
df["group"] = np.where(df["subject"].isin(healthy), False, True)
beta_df = pd.DataFrame(columns=states)

In [ ]:
df.head(5) # check out how the dataframe looks!

10. We then create a function that gives out the mean of the beta band for each run and each state. The dataframe "beta_df" then contains the beta band for each state of each run of each subject in each condition. We get it like this with the usage of our priorly defined function.

In [ ]:
def compute_beta_mean(data, idx_min, idx_max):
    try:
        return np.mean(np.asarray(data).mean(axis=0)[idx_min:idx_max])
    except IndexError:
        return None

In [ ]:
idx_max = processing.find_closest_index(ncs.freqs, 42)
idx_min = processing.find_closest_index(ncs.freqs, 20)


beta_df[states] = df[states].applymap(compute_beta_mean, na_action="ignore", idx_min=idx_min, idx_max=idx_max)
beta_df["group"] = df["group"]
beta_df["subject"] = df["subject"]
beta_df["run_id"] = df["date"] + "_" + df["run"]
beta_df["condition"] = df["condition"]

In [ ]:
beta_df.head(5) # check out how the dataframe looks!

11. Then, we create a new column that holds the mean of the three states for each row : 

In [ ]:
beta = beta_df[['event_fog', 'event_nlm', 'event_gait']].mean(axis=1, skipna=True)
beta_df["beta"] = beta

In [ ]:
beta_df.head(5) # check out how the dataframe looks!

12. Now, we group by condition and subject, so that we get the mean beta power for each condition of each animal. We also then drop columns we don't need and convert 0 & 1 into False and True, just to make the df a bit more neat : 

In [ ]:
grouped = beta_df.groupby(["subject", "condition"], as_index=False).mean()

In [ ]:
grouped.drop(["event_fog", "event_nlm", "event_gait", "run_id"], axis=1, inplace=True)
grouped["group"] = grouped["group"].astype(bool)

13. We create a dictionary to map the health status and the stimulation condition to one another. With this we can then create a new column called "identifyer". We need this for the seperation of different groups when plotting. 
For healthy animals we only want "Baseline" so every other condition for healthy animals gets assigned a NaN value. We can then drop those to filter out healthy animals with stimulation conditions that we don't want to plot :

In [ ]:
condition_map = {
    (False, "baseline"): "healthy_baseline",
    (True, "baseline"): "pda_baseline",
    (True, "cervical125"): "pda_cervical125",
    (True, "cervical40"): "pda_cervical40",
    (True, "lumbar125"): "pda_lumbar125",
    (True, "lumbar40"): "pda_lumbar40"
}

grouped["identifyer"] = grouped.apply(lambda row: condition_map.get((row["group"], row["condition"]), np.nan), axis=1)

In [ ]:
df = grouped.dropna(subset=["identifyer"], axis=0)

14. Let's define the colors for each group and the order of each group. After that we can plot the beta score plot : 
- There is also some code for making the plot look like I wanted it to. For example removing the edges of the box or making the median line thicker.
- With "chart" we also plot the mean beta value for the individual animals over our boxplot 

In [ ]:
color_palette = {"pda_baseline": "#e0493e", "pda_cervical40": "#93abd4", "pda_lumbar40": "#9ce29c", "pda_cervical125": "#496186", "pda_lumbar125": "#95b695", "healthy_baseline": "#808080"}
order = ["pda_baseline", "pda_lumbar40", "pda_lumbar125", "pda_cervical40", "pda_cervical125", "healthy_baseline"]

In [ ]:
plt.figure(figsize=(7, 4))
sns.set_style("ticks")
g = sns.boxplot(data=df, x="identifyer", y="beta", palette=color_palette, order=order, linewidth=0, medianprops=dict(linestyle='-', linewidth=2), capprops=dict(linestyle='-', linewidth=2), whiskerprops=dict(linestyle='-', linewidth=1))
chart = sns.scatterplot(data=df, y="beta", x="identifyer", color="k", legend=False, zorder=10, s=35)

g.spines[['right', 'top']].set_visible(False)
g.set_yticks([2, 5, 9])
g.set_xticks([])
plt.gca().set_xticklabels([])
plt.gca().set_xticklabels([])
g.set(ylabel=None, xlabel=None)
plt.tight_layout()
#plt.savefig("beta_boxplot_test.png", transparent=True)

15. You could also not plot the mean of all the states but plot each state or only the beta power in Akinesia. There are plenty of options, you just have to filter your dataframe differently.
16. Again, you can do a lot with the package "neurokin" that I have mostly used in this script. Check out https://neurokin.readthedocs.io/en/latest/index.html for a deep dive into the functions and for seeing what else you can do with it. 